In [54]:
using SeisPlot, SeisReconstruction, SeisProcessing
using  PyPlot, LinearAlgebra, FFTW, DSP, Statistics, LinearAlgebra

In [55]:
dt=0.002; # dt 
nt=512; #number of simples in time dimension
nx1=64; # number of samples in x1 dimension
nx2=64; # number of samples in x2 dimension
dx1=10.0; 
dx2=10.0;
f0= 25.0; # dominant frequency for the wavelet
tau=[0.256, 0.512, 0.768]; # t0 for the events
amp=[1.0, 0.5, -1.0]; # amplitude 
p2=[0.3,0.0, -0.45] ; # ray parameter dimension 1
p1=[0.15,0.0, -0.15]; # ray paramteter dimension2 


In [56]:

data=SeisParabEvents( tau=tau,amp=amp, p2=p2,p1=p1, dt=dt, nt=nt, dx1=dx1, nx1=nx1, dx2=dx2, nx2=nx2, f0=f0);

nt,nr,ns=size(data);

d_obs =(10^3)*copy(data); # scale



In [57]:
d_obsn= SeisAddNoise(d_obs, 1.0, db=false, L=9);


In [58]:
d_obsn=SeisDecimate(d_obsn;mode="random",perc=50);

In [62]:
S = CalculateSampling(d_obsn)

ρ=0.5;
μ= 5.0;
tol=1.e-4;
x0 = randn(Float64,size(d_obs));
operators=[ WeightingOp, FFTOp];
parameters= [Dict(:w =>S), Dict( :normalize=>true)];
m, J= ADMM(x0,d_obs,operators,parameters, ρ= ρ , μ=μ, Ne=100, Ni=100,tolin=1e-4, tolout=1e-4 );

UndefVarError: UndefVarError: `CalculateSampling` not defined

In [60]:
?ADDM

Base.Meta.ParseError: ParseError:
# Error @ /home/joaquin17/Desktop/PhD/PhD_codes/PhD_codes/SeisReconstruction.jl/examples/Demo_FISTA_reconstruction.ipynb:1:1
?ADDM
╙ ── not a unary operator

In [27]:
using Pkg

In [39]:
Pkg.status()

Project SeisReconstruction v0.1.2
Status `~/Desktop/PhD/PhD_codes/PhD_codes/SeisReconstruction.jl/Project.toml`
  [717857b8] DSP v0.7.9
⌅ [e30172f5] Documenter v0.27.25
  [35a29f4d] DocumenterTools v0.1.19
  [7a1cc6ca] FFTW v1.8.0
⌅ [a98d9a8b] Interpolations v0.14.7
  [37e2e46d] LinearAlgebra
  [de0858da] Printf
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
